In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import BarData

import datetime

In [2]:
class MyWrapper(EWrapper):

    def __init__(self):
        self.data = {}
        
    def nextValidId(self, orderId:int):
        #4 first message received is this one
        print("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        #5 start requests here
        self.start()

    def historicalData(self, reqId:int, bar: BarData):
        #7 data is received for every bar
        split = str(bar).split(' ')
        date_ = split[1][4:6] + '/' + split[1][6:8] + '/' + split[1][0:4]
        open_ = split[3]
        close_ = split[9]
        high_ = split[5]
        low_ = split[7]
        
        formatted = {'open':open_,'close':close_,'high':high_,'low':low_}
        self.data[date_] = formatted
        
#         print("HistoricalData. ReqId:", reqId, "BarData.", bar)

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        #8 data is finished
        global DATA
        print("HistoricalDataEnd. ReqId:", reqId, "from", start, "to", end)
        #9 this is the logical end of your program
        app.disconnect()
        print("finished")
        DATA = self.data

    def error(self, reqId, errorCode, errorString):
        # these messages can come anytime.
        print("Error. Id: " , reqId, " Code: " , errorCode , " Msg: " , errorString)

    def start(self):
        queryTime = (datetime.datetime.today() - datetime.timedelta(days=0)).strftime("%Y%m%d %H:%M:%S")

        contract = Contract()
        contract.symbol = "TSLA"
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.primaryExchange = "NASDAQ"
        
#         fx.secType = "CASH" 
#         fx.symbol = "USD"
#         fx.currency = "JPY"
#         fx.exchange = "IDEALPRO"

        #6 request data, using fx since I don't have Japanese data
        app.reqHistoricalData(4102, contract, queryTime,"6 M", "1 day", "MIDPOINT", 1, 1, False, [])
        
        app.reqFundamentalData(1, contract, "", [])

In [3]:
DATA = None

app = EClient(MyWrapper()) #1 create wrapper subclass and pass it to EClient
app.connect("127.0.0.1", 7496, clientId=123) #2 connect to TWS/IBG
data = app.run() #3 start message thread

Error. Id:  -1  Code:  502  Msg:  Couldn't connect to TWS. Confirm that "Enable ActiveX and Socket EClients" 
is enabled and connection port is the same as "Socket Port" on the 
TWS "Edit->Global Configuration...->API->Settings" menu. Live Trading ports: 
TWS: 7496; IB Gateway: 4001. Simulated Trading ports for new installations 
of version 954.1 or newer:  TWS: 7497; IB Gateway: 4002


In [4]:
DATA